# 🚢 Titanic classifier

🎯 In this challenge, the goal is to use SVM classifiers to predict whether a passenger survived or not (accuracy score), and compare your performance with your buddy of the day on an unseen test set that you will both share. Be aware that you will only have one trial on the test set!

In [17]:
import pandas as pd
import numpy as np
np.random.seed(8)

🚢 Import the `Titanic dataset`:

In [18]:
# Load dataset
import seaborn as sns
df = sns.load_dataset('titanic')

# Inspect basic info
df.head()
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


❓ **Question (Game Plan**) ❓ 

Write down below in plain english the different steps you are going to perform to answer the question.

#clean
split (training & test)
encode categorical var
scale numerical var
train a baseline svm model
perform hyperparametre tuning
evaluate perf

<details><summary>👨🏻‍🏫 <i>Read our answer suggested answer here</i></summary>
    
    
0. 🧹 Data Cleaning
1. ✂️ Train/Test Split
2. 🔡 Feature Encoding
3. ⚖️ Feature Scaling
4. 🐣 A first model
5. 🤖 Model Tuning: Cross-Validated RandomSearch (Coarse Grain approach first, Fine Grain afterwards)
6. 🕵🏻 True performance analysis on the test set
</details>

## (0) 🧹 Data Cleaning

❓ **Question (Duplicated rows)** ❓

Are there any duplicated rows ? If so, drop them.

In [19]:
df.duplicated().sum()

107

In [5]:
df = df.drop_duplicates()
df.duplicated().sum()  # vérification, doit afficher 0


0

❓ **Question (Missing values)** ❓

In which columns do we have missing values ?

Drop the column if there are too many missing values or impute these missing values.

In [9]:
# Check missing values
df.isna().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
embark_town    0
dtype: int64

In [8]:
# Drop 'deck' because it has too many missing values 
df = df.drop(columns=['deck'])

# Impute 'age' with median
df['age'] = df['age'].fillna(df['age'].median())

# Impute 'embarked' and 'embark_town' with mode
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

df = df.drop(columns=['alive', 'class', 'who', 'adult_male', 'alone'])


KeyError: "['alive', 'class', 'who', 'adult_male', 'alone'] not found in axis"

In [10]:
# Check again
df.isna().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
embark_town    0
dtype: int64

## (1) ✂️ Holdout

❓ **Question (Train-Test-Split)** ❓ 

* Holdout 30% of your dataset as the test set for a final evaluation  
    * Use `random_state=0` to compare your final results with your buddy's results)

In [11]:
from sklearn.model_selection import train_test_split

# Define X (features) and y (target)
X = df.drop(columns=['survived'])
y = df['survived']

# Train-test split (30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

# Check shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((548, 8), (236, 8), (548,), (236,))

## (2) 🔡 Encoding (the categorical variables)

✅ **Encoding the target**

👇 Your target is either `survived` or `died`. It was already done for you as shown down below.

In [12]:
data.survived.value_counts()

NameError: name 'data' is not defined

❓ **Question (Encoding the categorical features)** ❓

In [13]:
# Identify categorical features
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_features

['sex', 'embarked', 'embark_town']

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define the categorical and numerical features
categorical_features = ['sex', 'embarked', 'embark_town']
numerical_features = X_train.drop(columns=categorical_features).columns.tolist()

# Define the encoder
ohe = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore')

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)

# Fit on train only
preprocessor.fit(X_train)

# Apply transform
X_train_encoded = preprocessor.transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

# Check transformed shapes
X_train_encoded.shape, X_test_encoded.shape


((548, 12), (236, 12))

## (3) ⚖️ Feature Scaling

❓ **Question (Scaling)** ❓

Scale *both* your training set and your test set using the scaler of your choice

In [15]:

#Le scaler le plus classique pour un SVM = StandardScaler
#👉 centre les features (moyenne = 0) et les met à la même échelle (écart-type = 1)
#👉 indispensable pour un modèle basé sur distances comme SVM 

from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()

# Fit on train only
scaler.fit(X_train_encoded)

# Transform both train and test
X_train_scaled = scaler.transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Check shapes
X_train_scaled.shape, X_test_scaled.shape


((548, 12), (236, 12))

## (4) 🐣 Baseline Model

❓ **Question (Starting with a simple model...) ❓

Cross-validate a Linear SVC model as your baseline model, using the accuracy score. 

In [16]:
#Entraîner un LinearSVC simple
#Faire une cross-validation (par exemple cv=5)
#Mesurer la baseline accuracy
#Aucun tuning pour l’instant

from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# Instantiate baseline model
baseline_model = LinearSVC(random_state=0, max_iter=5000)

# Cross-validate on the TRAINING SET ONLY
scores = cross_val_score(
    baseline_model,
    X_train_scaled,
    y_train,
    cv=5,
    scoring='accuracy'
)

# Display results
print("Cross-validated scores:", scores)
print("Mean accuracy:", scores.mean())


Cross-validated scores: [0.80909091 0.73636364 0.74545455 0.75229358 0.77981651]
Mean accuracy: 0.7646038365304421


## (5) 🧨 Random Search

❓ **Question (Optimizing a Support Vector Classifier)** ❓

*  Use a **RandomizedSearchCV** to optimize both the parameters `kernel` and `C` of an SVM classifier (SVC)
    - Start with a total of `n_iter=100` combinations, cross-validated `cv=5` times each
    - Use `verbose=1`to check progress
    - Use `n_jobs=-1` to use all your CPU cores
    - (Optional) You can also optimize other parameters of your choice if you want to.

☣️ If the `RandomizedSearchCV` seems stuck after more than 10 seconds, perform one search per SVM kernel. Scikit-Learn sometimes experiences issues with _Searching_ multiple kernels at a time

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

models = {
    "LogisticRegression": LogisticRegression(max_iter=500, n_jobs=-1),
    "LinearSVC": LinearSVC(max_iter=5000),
    "SVC_rbf": SVC(kernel='rbf'),
    "RandomForest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier()
}

results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    results[name] = scores
    print(f"{name} -> mean: {scores.mean():.4f} | std: {scores.std():.4f} | scores: {scores}")

# sort and display
print("\n=== Sorted by mean accuracy ===")
for name, scores in sorted(results.items(), key=lambda x: x[1].mean(), reverse=True):
    print(f"{name}: {scores.mean():.4f}")


❓ **Question(Optimized Model and its performance)** ❓

* What are the best parameters ?
* What is the best score?

In [ ]:
# YOUR CODE HERE

## (6) 🕵️‍♀️ Final test score and Confusion Matrix

❓ **Question (Evaluating on the test set)** ❓

* Select the best model you want to test. You will compare your result with your buddy of the day!

* Compute its `accuracy`, `classification_report` and show the `confusion_matrix` on the test set.

☣️ You can only test one model. Once you have seen the test set, any other optimization would result in data leakage 

In [ ]:
# YOUR CODE HERE

❓ **Question (Confusion Matrix)** ❓

In [ ]:
# YOUR CODE HERE

🏁 Congratulations! You were able to tackle a classification task from A to Z, cleaning your dataset, encoding and scaling your features, optimizing your model... !

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!